In [1]:
import zipfile
import pandas as pd

## Unzip Cluster Dataset

In [2]:
# Percorso del file ZIP da estrarre
zip_file_path = "real-time clusters/20230101-20240101_real_time_clusters.parquet.zip"

# Percorso della cartella di destinazione per l'estrazione
extract_to_folder = "real-time clusters"

# Aprire il file ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Estrarre tutto il contenuto nella cartella di destinazione
    zip_ref.extractall(extract_to_folder)

In [2]:
real_time_clusters_df = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")

In [3]:
real_time_clusters_df.columns

Index(['alarm_id', 'network', 'ne_type', 'first_occurrence', 'last_update',
       'data', 'ne_id', 'geo_code', 'min_cluster_first_occurrence',
       'link_cluster', 'domain', 'cluster_id', 'cluster_id2'],
      dtype='object')

In [4]:
real_time_clusters_df["network"].unique()

array(['an', 'sdh', 'ptn', 'adsl'], dtype=object)

## Filter Clusters
- At least 3 alarms inside a cluster
- At least 2 different devices inside a cluster

In [5]:
counts = real_time_clusters_df['cluster_id2'].value_counts()
unique_ne_ids = real_time_clusters_df.groupby('cluster_id2')['ne_id'].nunique()

filtered_df = real_time_clusters_df[
    (real_time_clusters_df['cluster_id2'].isin(counts[counts >= 3].index)) &
    (real_time_clusters_df['cluster_id2'].isin(unique_ne_ids[unique_ne_ids > 1].index))
]

In [6]:
filtered_df.groupby('cluster_id2').size().sort_values(ascending=False)

cluster_id2
202309181126_17    10079
202308080334_43     7989
202311270204_5      6736
202311270406_8      6618
202310110506_21     6226
                   ...  
202309070948_8         3
202309070950_21        3
202309070958_3         3
202309070958_8         3
202312312358_14        3
Length: 816089, dtype: int64

In [7]:
filtered_df.to_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")